In [1]:
from numpy import mat,shape,nonzero,eye
import numpy.linalg as la

In [2]:
def loadExtData():
    # mat A
    return [[4,4,0,2,2],
            [4,0,0,3,3],
            [4,0,0,1,1],
            [1,1,1,2,0],
            [2,2,2,0,0],
            [1,1,1,0,0],
            [5,5,5,0,0]]

In [3]:
dataMat = loadExtData()
U,S,V = la.svd(dataMat)
print(shape(U))
print(shape(S))
print(shape(V))
print(S,"\n")

(7, 7)
(5,)
(5, 5)
[11.90341619  5.99130503  2.59510658  1.92662869  0.98309238] 



In [4]:
S_3 = mat([[S[0],0,0],[0,S[1],0],[0,0,S[2]]])
restoreData = U[:,:3]*S_3*V[:3,:]
print(shape(restoreData))
print(restoreData)

(7, 5)
[[ 3.75882897e+00  3.93552733e+00  3.14268322e-01  2.41213328e+00
   2.03313584e+00]
 [ 4.37098660e+00  7.04080744e-02 -4.62953777e-01  2.71367176e+00
   2.58910050e+00]
 [ 3.51434619e+00 -9.90889288e-02  6.10971280e-01  1.45843497e+00
   1.45134300e+00]
 [ 1.42793888e+00  1.16461243e+00  4.06617551e-01  6.61942661e-01
   5.69400307e-01]
 [ 2.02506118e+00  2.00314495e+00  1.96987305e+00  1.29268899e-04
  -4.79169143e-02]
 [ 1.01253059e+00  1.00157247e+00  9.84936526e-01  6.46344496e-05
  -2.39584571e-02]
 [ 5.06265296e+00  5.00786237e+00  4.92468263e+00  3.23172248e-04
  -1.19792286e-01]]


In [5]:
def cos_sim(inA,inB):
    tmp = float(inA.T * inB)
    dis = 0.5+0.5*tmp/(la.norm(inA)*la.norm(inB))
    return dis

In [6]:
def loadExtData2():
    return [[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
            [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
            [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
            [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
            [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
            [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
            [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
            [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
            [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
            [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
            [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [26]:
def svdEst(dataMat, user, simMeas, item, k):
    if(dataMat[user, item] != 0): 
        return dataMat[user, item]
    n = shape(dataMat)[1] 
    simTotal = 0.0;
    ratSimTotal = 0.0
    # SVD分解
    U, S, V = la.svd(dataMat)
    # 降维，获取前k个特征值
    S3 = mat(eye(k) * S[:k])
    # 对item矩阵进行降维操作，把V从（11，11）降到（11，k），推导可参考后面
    xformedItems = dataMat.T * U[:, :k] * S3.I
    print(shape(xformedItems))
    for j in range(n):
        userRating = dataMat[user, j]
        if(userRating == 0 or j == item): 
            continue
        # 计算未评分item和已评分j的相似度
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # 加权
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if(simTotal == 0):  
        return 0
    else:   
        return ratSimTotal / simTotal

In [27]:
def svdRecommend(dataMat, user, N=3, simMean=cos_sim, estTestMean=svdEst, k=3 ):
    unRatedItem = nonzero(dataMat[user,:].A == 0)[1] # .A change matrix to array
    if(len(unRatedItem) == 0):print('There is nothing to recommend')
    retScores = [] # scores of unRatedItems
    for item in unRatedItem:
        itemScore=estTestMean(dataMat, user, simMean, item, k=k) # predicton of user for item
        retScores.append((item, itemScore))
    return sorted(retScores, key=lambda j:j[1], reverse=True)[:N]

In [28]:
myData = mat(loadExtData2())
U, S, V = la.svd(myData)
S *= S
threshold = sum(S) * 0.9
k = 0

for i in range(S.shape[0]+1):
    if(sum(S[:i]) >= threshold):
        k = i
        break

In [29]:
svdItems = svdRecommend(myData, user=3, estTestMean=svdEst,k=k)
print(svdItems)

(11, 3)
[[-0.00965352 -0.34029577 -0.2272107 ]]
[[-0.00965352 -0.34029577 -0.2272107 ]]
[[-0.00965352 -0.34029577 -0.2272107 ]]
[[-0.00965352 -0.34029577 -0.2272107 ]]
[[-0.00965352 -0.34029577 -0.2272107 ]]
(11, 3)
[[-0.01525086 -0.38739367  0.61205988]]
[[-0.01525086 -0.38739367  0.61205988]]
[[-0.01525086 -0.38739367  0.61205988]]
[[-0.01525086 -0.38739367  0.61205988]]
[[-0.01525086 -0.38739367  0.61205988]]
(11, 3)
[[-0.01596356 -0.52009013 -0.36077717]]
[[-0.01596356 -0.52009013 -0.36077717]]
[[-0.01596356 -0.52009013 -0.36077717]]
[[-0.01596356 -0.52009013 -0.36077717]]
[[-0.01596356 -0.52009013 -0.36077717]]
(11, 3)
[[-0.00454354 -0.18887353 -0.00920021]]
[[-0.00454354 -0.18887353 -0.00920021]]
[[-0.00454354 -0.18887353 -0.00920021]]
[[-0.00454354 -0.18887353 -0.00920021]]
[[-0.00454354 -0.18887353 -0.00920021]]
(11, 3)
[[-0.01548618 -0.39195392  0.5571563 ]]
[[-0.01548618 -0.39195392  0.5571563 ]]
[[-0.01548618 -0.39195392  0.5571563 ]]
[[-0.01548618 -0.39195392  0.5571563 ]]
